# Action Graph 

### Goals:

- A graph store/database to store and trace any computations during eager execution
- Graph that works with the current in memory worker
- Ability to visualize the graph
- Generate a dependecy list of node, so that any dependeny action can be generated
- Basic query/search functionalities
- Locking/Concurrency

In [ ]:
import syft as sy
from syft.service.action.action_graph_service import ActionGraphService, NodeActionDataUpdate, ActionStatus
from syft.service.action.action_graph import InMemoryActionGraphStore, InMemoryGraphConfig, InMemoryStoreClientConfig

from syft.service.context import AuthedServiceContext
from syft.node.credentials import SyftSigningKey
from syft.service.action.action_graph import Action
from syft.service.action.numpy import NumpyArrayObject, ActionObject
import numpy as np
import matplotlib.pyplot as plt

## Scenario for performing some computation

```python

import syft as sy

domain_client = sy.login("....")

dataset = domain_client.datasets[0]

a = dataset.assets["A"]

b = dataset.assets["B"]

c = a + b

d = domain_client.api.numpy.array([1, 2, 3])

e = c * d

# Inplace operation, mutated the value of d
d.astype('int32')

f = d + 48
```

<br>
<br>

**Corresponding Actions Generated**

```

action1 -> a + b

action2 -> initialization of variable `d`

action3 -> c * d

action4 -> inplace updation of type of `d` (d.astype('int32'))

action5 -> d + 48

```

![graph.png](graph.png)

### Initializing the Store

In [ ]:
# Create a Config

store_config = InMemoryGraphConfig()

In [ ]:
# Initialize the InMemory Store

graph_store = InMemoryActionGraphStore(store_config=store_config)

### Initializing Action Graph Service

In [ ]:
action_graph_service = ActionGraphService(store=graph_store)

In [ ]:
signing_key = SyftSigningKey.generate()
authed_context = AuthedServiceContext(credentials=signing_key.verify_key)

In [ ]:
signing_key = SyftSigningKey.generate()
authed_context = AuthedServiceContext(credentials=signing_key.verify_key)

### Create some dummy data

In [ ]:
action_obj_a = ActionObject.from_obj([2, 4, 6])
action_obj_b = ActionObject.from_obj([2, 3, 4])

In [ ]:
action_obj_a.id, action_obj_b.id

### Action1 -> A + B

In [ ]:
action1 = Action(
    path="action.execute",
    op="__add__",
    remote_self=action_obj_a.syft_lineage_id,
    args=[action_obj_b.syft_lineage_id],
    kwargs={}
)
action1

### Save action to Graph

In [ ]:
action_graph_service.add_action

In [ ]:
action_graph_service.add_action(context=authed_context, action=action1)

### Action2 -> np.array([1, 2, 3])

In [ ]:
action_obj_d = ActionObject.from_obj([1, 2, 3])

In [ ]:
# Create Action2

action2 = Action(
    path="action.execute",
    op="np.array",
    remote_self=None,
    args=[action_obj_d.syft_lineage_id],
    kwargs={}
)
action2

In [ ]:
# Save action to graph
action_graph_service.add_action(context=authed_context, action=action2)

In [ ]:
# 747749f9494345b78e165f13351e52bf: {"data": NodeActionData()}

In [ ]:
plt.figure(figsize=(20, 10))
action_graph_service.store.graph.visualize()

### Action3 -> C * D

In [ ]:
action3 = Action(
    path="action.execute",
    op="__mul__",
    remote_self=action1.result_id,
    args=[action2.result_id],
    kwargs={}
)
action3

In [ ]:
action_graph_service.add_action(context=authed_context, action=action3)

In [ ]:
plt.figure(figsize=(20, 10))
action_graph_service.store.graph.visualize()

### Action4 -> Mutate type of D

In [ ]:
as_type_action_obj = ActionObject.from_obj('np.int32')

In [ ]:
as_type_action_obj

In [ ]:
action4 = Action(
    path="action.execute",
    op="astype",
    remote_self=action2.result_id,
    args=[as_type_action_obj.syft_lineage_id],
    kwargs={},
    result_id=action2.result_id
)
action4

In [ ]:
action_graph_service.add_action(context=authed_context, action=action4)

In [ ]:
plt.figure(figsize=(20, 10))
action_graph_service.store.graph.visualize()

In [ ]:
action_2_node_data = action_graph_service.store.get(uid=action2.id, credentials=authed_context.credentials).ok()
action_2_node_data

In [ ]:
action_graph_service.update(context=authed_context, action_id=action2.id, node_data=NodeActionDataUpdate(is_mutated=True))

In [ ]:
action_graph_service.store.get(uid=action2.id, credentials=authed_context.credentials).ok()

In [ ]:
plt.figure(figsize=(20, 10))
action_graph_service.store.graph.visualize()

### Action5 -> D + 48

In [ ]:
arg_action_obj = ActionObject.from_obj(48)

In [ ]:
# action2.result_id == action4.result_id

In [ ]:
action5 = Action(
    path="action.execute",
    op="__add__",
    remote_self=action2.result_id,
    args=[arg_action_obj.syft_lineage_id],
    kwargs={},
)
action5

In [ ]:
action_graph_service.add_action(context=authed_context, action=action5)

In [ ]:
plt.figure(figsize=(20, 10))
action_graph_service.store.graph.visualize()

In [ ]:
action1.id, action2.id, action3.id, action4.id, action5.id

```

action1 -> a + b

action2 -> initialization of variable `d`

action3 -> c * d

action4 -> inplace updation of type of `d` (d.astype('int32'))

action5 -> d + 48

```

## Filtering Actions in the Graph

### Filter by ActionStatus

```
ActionStatus
- PROCESSING
- DONE
- FAILED
```

In [ ]:
action_graph_service.get_by_action_status(context=authed_context, status=ActionStatus.PROCESSING)

In [ ]:
action_graph_service.get_by_action_status(context=authed_context, status=ActionStatus.DONE)

### Filter by Particular User

In [ ]:
action_graph_service.get_by_verify_key(context=authed_context, verify_key=signing_key.verify_key)

In [ ]:
### Serde 

In [ ]:
# bytes_data = sy.serialize(graph_store, to_bytes=True)

In [ ]:
# graph_store = sy.deserialize(bytes_data, from_bytes=True)